<a href="https://colab.research.google.com/github/hyeeunun/-/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%98%EB%B0%8D_%EC%A3%BC%EC%A0%9C_%EC%84%B1%EA%B2%A9_%EC%9C%A0%ED%98%95_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'predict-people-personality-types:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5622628%2F9392095%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240922%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240922T154227Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da3c671648789dd4242bebb98e030f79345fd6c6d689394e99b21c7db9671de2a9c45687fb89c44cd8a522e223762c4e8723e98bf9d5641da635fbe9e75d7115cf7dec59f68eec5eb38dcea83c3404d887164d8eace282efc84235d5e566e161797193f5fe06035a661cfac116584c0347236784036e7f129c8aaea5fb3250fbc235b8f2e418a78d0e218342314ea053bd7a2d86e882ca9f408b65e984c8088efbfd8bdb0a75cb552c3587bd0eda291428ef0891e30451d71c0cb37159e931b2caf7dfd5ab64a054bcac56637fc87e34599341d3df77f0b713ad73b3c95f858bebf737a9fcf3463decfdbf8d6b1e20a6bcaff53cc567506641b7828cb6bf7f263'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd
import plotly.express as px
from ydata_profiling import ProfileReport


In [ ]:
df = pd.read_csv('/kaggle/input/predict-people-personality-types/data.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.describe()

#  Feature Descriptions
* Age: A continuous variable representing the age of the individual.
*
* Gender: A categorical variable indicating the gender of the individual. Possible values are 'Male' and 'Female'.
*
* Education: A binary variable, A value of 1 indicates the individual has at least a graduate-level education (or higher), and 0 indicates an undergraduate, high school level or Uneducated.
*
* Interest: A categorical variable representing the individual's primary area of interest.
*
* Introversion Score: A continuous variable ranging from 0 to 10, representing the individual's tendency toward introversion versus extraversion. Higher scores indicate a greater tendency toward extraversion.
*
* Sensing Score: A continuous variable ranging from 0 to 10, representing the individual's preference for sensing versus intuition. Higher scores indicate a preference for sensing.
*
* Thinking Score: A continuous variable ranging from 0 to 10, indicating the individual's preference for thinking versus feeling. Higher scores indicate a preference for thinking.
*
* Judging Score: A continuous variable ranging from 0 to 10, representing the individual's preference for judging versus perceiving. Higher scores indicate a preference for judging.
*
* Personality: Target that contains People Personality Type

In [ ]:
report = ProfileReport(df , dark_mode= True  , title = 'EDA REPORT OF THE DATA')
report

 # Here is a brief description of each of the 16 Myers-Briggs Type Indicator (MBTI) personality types you mentioned:

**ENFP (Extraverted, Intuitive, Feeling, Perceiving):**

Enthusiastic, creative, and sociable free spirits who see life as full of possibilities. ENFPs are highly independent and prefer novelty and excitement.

**ESFP (Extraverted, Sensing, Feeling, Perceiving):**

Fun-loving, spontaneous, and energetic individuals who enjoy being the center of attention. ESFPs are highly in tune with their surroundings and love to entertain.

**INTP (Introverted, Intuitive, Thinking, Perceiving):**

Analytical, logical, and inventive thinkers who enjoy deep thought and solving complex problems. INTPs value independence and intellectual exploration.

**INFP (Introverted, Intuitive, Feeling, Perceiving):**

Idealistic, sensitive, and compassionate individuals who strive for harmony and understanding. INFPs focus on personal values and love creative expression.

**ENFJ (Extraverted, Intuitive, Feeling, Judging):**

Charismatic and empathetic leaders who help others grow and develop. ENFJs are organized, persuasive, and skilled at reading emotions.

**ENTP (Extraverted, Intuitive, Thinking, Perceiving):**

Quick-witted and resourceful, ENTPs enjoy debates and new ideas. They thrive on intellectual challenges and are always seeking innovation.

**ESTP (Extraverted, Sensing, Thinking, Perceiving):**

Action-oriented, adaptable, and pragmatic individuals who enjoy taking risks and living in the moment. ESTPs excel in crisis situations and love excitement.

**ISTP (Introverted, Sensing, Thinking, Perceiving):**

Practical and logical problem solvers who love working with their hands and understanding how things work. ISTPs are independent and spontaneous.

**INTJ (Introverted, Intuitive, Thinking, Judging):**

Strategic, logical, and future-oriented individuals who excel at planning and organizing. INTJs are often visionaries, valuing knowledge and competence.

**INFJ (Introverted, Intuitive, Feeling, Judging):**

Insightful and empathetic individuals with a strong sense of morality. INFJs focus on helping others and often feel called to serve a higher purpose.

**ISFP (Introverted, Sensing, Feeling, Perceiving):**

Gentle, artistic, and sensitive individuals who appreciate beauty and value personal freedom. ISFPs tend to be quiet and prefer hands-on experiences.

**ENTJ (Extraverted, Intuitive, Thinking, Judging):**

Bold, confident, and decisive leaders who excel at organizing and achieving their goals. ENTJs are highly strategic and value efficiency.

**ESFJ (Extraverted, Sensing, Feeling, Judging):**

Warm, friendly, and organized individuals who enjoy helping others and maintaining harmony. ESFJs value tradition and are often highly sociable.

**ISFJ (Introverted, Sensing, Feeling, Judging):**

Caring, loyal, and detail-oriented individuals who enjoy helping others and creating a sense of stability. ISFJs are highly dependable and organized.

**ISTJ (Introverted, Sensing, Thinking, Judging):**

Practical, responsible, and thorough individuals who value traditions and structure. ISTJs are reliable and prefer working in established systems.

**ESTJ (Extraverted, Sensing, Thinking, Judging):**

Efficient, organized, and take-charge individuals who excel at managing people and projects. ESTJs are highly logical and value order.

In [ ]:
persona = df.groupby('Personality')['Gender'].value_counts().sort_values(ascending = False).reset_index()


fig  = px.bar(persona , y = 'count' , x= 'Personality' , color = 'Gender' , text_auto=True , title = 'GENDERS COUNT IN EVERY KIND OF PERSONALITY', height = 600)

fig.show()

In [ ]:
intr = df.groupby('Personality')['Interest'].value_counts().sort_values(ascending = False).reset_index()

fig  = px.bar(intr , y = 'count' , x= 'Personality' , color = 'Interest' , text_auto=True , title = 'INTEREST COUNT IN EVERY KIND OF PERSONALITY', height = 600)

fig.show()


In [ ]:
# Function to categorize age into brackets
def age_bracket(age):
    if 18 <= age < 30:
        return '20s'
    elif 30 <= age < 40:
        return '30s'
    elif 40 <= age < 50:
        return '40s'
    else:
        return '50s'

# Apply the function to create a new column 'Age Bracket'
df['Age Bracket'] = df['Age'].apply(age_bracket)

In [ ]:
ag = df.groupby('Gender')['Age Bracket'].value_counts().sort_values(ascending = False).reset_index()

ag.iloc[: , [1,0,2]]

In [ ]:
age_bracet = df.groupby('Personality')[['Gender','Age Bracket']].value_counts().sort_values(ascending = False).reset_index()

fig  = px.bar(age_bracet , y = 'count' , x= 'Personality' , color = 'Age Bracket' ,hover_data='Gender' , text_auto=True , title = 'Age Bracket COUNT IN EVERY KIND OF PERSONALITY', height = 800)

fig.show()


# ****THANK YOU..........****